In [1]:
from scipy.stats import norm

In [2]:
import torch

In [3]:
import numpy as np

In [4]:
norm.ppf(0.95)

1.6448536269514722

In [243]:
DEVICE = 'cpu'
size = 6
taus = [(2 * i + 1) / (2 * size)  for i in range(size)]
print(taus)
v = torch.tensor([19., 1., 1., 1.3, 1.5, 2], dtype=torch.float32)
# v = torch.ones((size,), dtype=torch.float32)
# v = [1, 1, 1]
Q = torch.tensor([[9], [10.3], [12.2], [13.2], [15.2], [19]], dtype=torch.float32).unsqueeze(0)
# Q = torch.tensor([[9], [10.3], [12.2], [13.2], [15.2], [19], [20.3], [22.2], [23.2], [25]], dtype=torch.float32).unsqueeze(0)
# X = torch.tensor([[1, norm.ppf(taus[0])], [1, norm.ppf(taus[1])], [1, norm.ppf(taus[2])]], dtype=torch.float32).unsqueeze(0)
quantile_tau = taus 
X = [[1, norm.ppf(t), norm.ppf(t)**2 - 1, norm.ppf(t)**3 - 3 * norm.ppf(t)] for t in quantile_tau]
# X = [[1, norm.ppf(t), (norm.ppf(t)**2 - 0.3)] for t in quantile_tau]
X = torch.tensor(X, device=DEVICE).float()
V = torch.diag(v)
V = V.unsqueeze(0)

[0.08333333333333333, 0.25, 0.4166666666666667, 0.5833333333333334, 0.75, 0.9166666666666666]


In [244]:

Q, X, V

(tensor([[[ 9.0000],
          [10.3000],
          [12.2000],
          [13.2000],
          [15.2000],
          [19.0000]]]),
 tensor([[ 1.0000, -1.3830,  0.9127,  1.5038],
         [ 1.0000, -0.6745, -0.5451,  1.7166],
         [ 1.0000, -0.2104, -0.9557,  0.6220],
         [ 1.0000,  0.2104, -0.9557, -0.6220],
         [ 1.0000,  0.6745, -0.5451, -1.7166],
         [ 1.0000,  1.3830,  0.9127, -1.5038]]),
 tensor([[[19.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  1.0000,  0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  1.3000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  0.0000,  1.5000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  2.0000]]]))

In [245]:
Q, X, V = Q.expand(2, -1, -1), X.expand(2, -1, -1), V.expand(2, -1, -1)
Q, X, V

(tensor([[[ 9.0000],
          [10.3000],
          [12.2000],
          [13.2000],
          [15.2000],
          [19.0000]],
 
         [[ 9.0000],
          [10.3000],
          [12.2000],
          [13.2000],
          [15.2000],
          [19.0000]]]),
 tensor([[[ 1.0000, -1.3830,  0.9127,  1.5038],
          [ 1.0000, -0.6745, -0.5451,  1.7166],
          [ 1.0000, -0.2104, -0.9557,  0.6220],
          [ 1.0000,  0.2104, -0.9557, -0.6220],
          [ 1.0000,  0.6745, -0.5451, -1.7166],
          [ 1.0000,  1.3830,  0.9127, -1.5038]],
 
         [[ 1.0000, -1.3830,  0.9127,  1.5038],
          [ 1.0000, -0.6745, -0.5451,  1.7166],
          [ 1.0000, -0.2104, -0.9557,  0.6220],
          [ 1.0000,  0.2104, -0.9557, -0.6220],
          [ 1.0000,  0.6745, -0.5451, -1.7166],
          [ 1.0000,  1.3830,  0.9127, -1.5038]]]),
 tensor([[[19.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  1.0000,  0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,

In [246]:
Q.shape, X.shape, V.shape

(torch.Size([2, 6, 1]), torch.Size([2, 6, 4]), torch.Size([2, 6, 6]))

In [247]:
(X.transpose(1, 2).bmm(V.inverse()).bmm(X)).inverse().bmm(X.transpose(1, 2)).bmm(V.inverse()).bmm(Q)

tensor([[[13.1681],
         [ 4.0554],
         [ 0.5584],
         [ 0.2101]],

        [[13.1681],
         [ 4.0554],
         [ 0.5584],
         [ 0.2101]]])

In [248]:
(X.transpose(1, 2).bmm(X)).inverse().bmm(X.transpose(1, 2)).bmm(Q)

tensor([[[13.2948],
         [ 3.7160],
         [ 0.7385],
         [ 0.0861]],

        [[13.2948],
         [ 3.7160],
         [ 0.7385],
         [ 0.0861]]])

In [236]:
Q.mean()

tensor(13.1500)

In [249]:
(X.T.mm(V.inverse()).mm(X)).inverse().mm(X.T).mm(V.inverse()).mm(Q)

RuntimeError: self must be a matrix

In [252]:
torch.arange(2*3*5).reshape(2, 3, 5).float()

tensor([[[ 0.,  1.,  2.,  3.,  4.],
         [ 5.,  6.,  7.,  8.,  9.],
         [10., 11., 12., 13., 14.]],

        [[15., 16., 17., 18., 19.],
         [20., 21., 22., 23., 24.],
         [25., 26., 27., 28., 29.]]])

In [254]:
torch.arange(2*3*5).reshape(2, 3, 5).float().std(1)

tensor([[5., 5., 5., 5., 5.],
        [5., 5., 5., 5., 5.]])